# Amazon Bedrock Claude3 Caculator_tool

- https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html
- https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/calculator_tool.ipynb

In [1]:
!pip install -U boto3 botocore

## 1. Bedrock 호출 함수 선언

In [1]:
from botocore.exceptions import ClientError


class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass


def calculate(expression):
    import re
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"
    

def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])


def claude_converse(messages, model='haiku', stream=True, system=None, toolConfig=None, region_name='us-west-2'):
    import boto3
    from botocore.config import Config
    
    config = Config(
        read_timeout=600, ## Timeout 시간 조정
        retries = dict(
            max_attempts = 8 ## Retry 횟수 조정
        )
    )

    # Create a Bedrock Runtime client in the AWS Region of your choice.
    client = boto3.client("bedrock-runtime", region_name=region_name, config=config)
    
    params = {}
    
    params['messages'] = messages
    
    if model == 'opus':
        params['modelId'] = "anthropic.claude-3-opus-20240229-v1:0"
    elif model == 'sonnet':
        params['modelId'] = "anthropic.claude-3-sonnet-20240229-v1:0"
    else:
        params['modelId'] = "anthropic.claude-3-haiku-20240307-v1:0"
    
    
    # Format the request payload using the model's native structure.
    params['inferenceConfig'] = {
        "maxTokens": 512,
        "temperature": 0.5,
        "topP": 0.999
    }
    
    ## Additional inference parameters that the model supports
    params['additionalModelRequestFields'] = {"top_k": 350}
    
    if system:
        params['system'] = [{"text" : system}]
    
    # print(f"tools : {tools}")
    if toolConfig:
        params['toolConfig'] = toolConfig
    
    if stream:
        response = client.converse_stream(**params)
    else:
        response = client.converse(**params)
    return response

    
    
def chat_with_claude_converse_tooluse(user_message, model='haiku', stream=True, system=None, toolConfig=None, region_name='us-west-2'):
    
    messages = [{
        "role": "user",
        "content": [{"text": user_message}]
    }]

    response = claude_converse(messages, model, stream, system, toolConfig, region_name)
    
    if stream:
        return_params = tool_use(response)
        
        role = return_params.get('role')
        toolUseId = return_params.get('toolUseId')
        tool_name = return_params.get('tool_name')
        tool_input = return_params.get('tool_input')
        text_input = return_params.get('text_input')
        pre_inputTokens = return_params.get('inputTokens')
        pre_outputTokens = return_params.get('outputTokens') 
        pre_totalTokens = return_params.get('totalTokens')
        pre_latencyMs = return_params.get('latencyMs')
        stop_reason = return_params.get('stop_reason')
        
        msg_dict = {}
        msg_dict['toolUse'] = {}
        
        if text_input:
            msg_dict['text']= text_input
        if toolUseId:
            msg_dict['toolUse']['toolUseId']= toolUseId
        if tool_name:
            msg_dict['toolUse']['name']= tool_name
        if tool_input:
            msg_dict['toolUse']['input']= tool_input
        
        messages.append(
            {
                "role": role,
                "content": [msg_dict]
            }
        )
        
        if stop_reason == 'tool_use':
            if tool_name == 'calculator':
                print(f"Requesting tool {tool_name}. Request: {toolUseId}")
                tool_result = {}
                try:
                    result = process_tool_call(tool_name, tool_input)
                    print(f"result : {result}")

                    tool_result = {
                        "toolUseId": toolUseId,
                        "content": [{"json": {"result": result}}]
                    }
                except StationNotFoundError as err:
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text":  err.args[0]}],
                        "status": 'error'
                    }

                tool_result_message = {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": tool_result

                        }
                    ]
                }
                messages.append(tool_result_message)
                # print(f"messages : {messages}")
                # Send the tool result to the model.
                response = claude_converse(messages, model, stream, system, toolConfig, region_name)
                return_params = tool_use(response, pre_inputTokens, pre_outputTokens, pre_totalTokens, pre_latencyMs, last_output=True)
 
                        
        # print the final response from the model.
        for content in output_message['content']:
            if content.get("text"):
                print(f"Text: {content['text']}")
            elif content.get("toolUse"):
                print(f"toolUseId: {content['toolUse']['toolUseId']}")
                print(f"name: {content['toolUse']['name']}")
                print(f"input: {content['toolUse']['input']}")
            
        print(f"Input tokens:  {inputTokens}")
        print(f"Output tokens:  {outputTokens}")
        print(f"Total tokens:  {totalTokens}")
        print(f"Stop reason: {response['stopReason']}")


def output_claude_converse_stream(messages, model='haiku',system=None, toolConfig=None, region_name='us-west-2'):
    from botocore.exceptions import ClientError
    import json
    msg_param = {}
    msg_param['text_input'] = ""
    msg_param['tool_input'] = ""

    try:
        response = claude_converse(messages, model, True, system, toolConfig, region_name)
        msg_tooluse = ""
        msg_print = False
        stream_response = response.get('stream')
        for event in stream_response:
            if 'messageStart' in event:
                msg_param['role'] = event['messageStart']['role']
                print(f"\nRole: {event['messageStart']['role']}")

            if 'contentBlockStart' in event:
                contentblock = event['contentBlockStart']['start']
                if contentblock.get("toolUse"):
                    msg_param['toolUseId'] = contentblock['toolUse']['toolUseId']
                    msg_param['tool_name'] = contentblock['toolUse']['name']
            
            if 'contentBlockDelta' in event:
                delta = event['contentBlockDelta']['delta']
                if delta.get('text'):
                    msg_param['text_input'] += event['contentBlockDelta']['delta']['text']
                    print(event['contentBlockDelta']['delta']['text'], end="")
                elif delta.get('toolUse'):
                    msg_param['tool_input'] += event['contentBlockDelta']['delta']['toolUse']['input']
                    print(event['contentBlockDelta']['delta']['toolUse']['input'], end="")
            if 'messageStop' in event:
                msg_param['stop_reason'] = event['messageStop']['stopReason']
                print(f"\nStop reason: {event['messageStop']['stopReason']}")

            if 'metadata' in event:
                metadata = event['metadata']
                if 'usage' in metadata:
                    print("\nToken usage")
                    print(f"Input tokens: {metadata['usage']['inputTokens']}")
                    print(
                        f"Output tokens: {metadata['usage']['outputTokens']}")
                    print(f"Total tokens: {metadata['usage']['totalTokens']}")
                if 'metrics' in event['metadata']:
                    print(
                        f"Latency: {metadata['metrics']['latencyMs']} milliseconds")
    except ClientError as err:
        message = err.response['Error']['Message']
        print("A client error occurred: %s", message)

    else:
        print(
            f"\nFinished generating text with model {model}.")
    return response, msg_param


def caculator_stream_output(messages, response, msg_param):
    import json

    stream_response = response.get('stream')
    role = msg_param['role']
    stop_reason = msg_param['stop_reason']

    msg_dict = {}
    msg_dict['toolUse'] = {}
    
    toolUseId , tool_name, tool_input = "", "", ""

    if msg_param.get('tool_input'):
        tool_input = msg_param['tool_input']
        tool_input = json.loads(tool_input)
        msg_dict['toolUse']['input']= tool_input
    if msg_param.get('text'):
        msg_dict['text']= text_input
    if msg_param.get('toolUseId'):
        toolUseId = msg_param['toolUseId']
        msg_dict['toolUse']['toolUseId']= toolUseId
    if msg_param.get('tool_name'):
        tool_name = msg_param['tool_name']
        msg_dict['toolUse']['name']= tool_name

    messages.append(
        {
            "role": role,
            "content": [msg_dict]
        }
    )
    
    if stop_reason == 'tool_use':
        if tool_name == 'calculator':
            print(f"Requesting tool {tool_name}. Request: {toolUseId}")
            tool_result = {}
            try:
                result = process_tool_call(tool_name, tool_input)
                print(f"result : {result}")

                tool_result = {
                    "toolUseId": toolUseId,
                    "content": [{"json": {"result": result}}]
                }
            except StationNotFoundError as err:
                tool_result = {
                    "toolUseId": tool['toolUseId'],
                    "content": [{"text":  err.args[0]}],
                    "status": 'error'
                }
            
            tool_result_message = {
                "role": "user",
                "content": [
                    {
                        "toolResult": tool_result

                    }
                ]
            }
            messages.append(tool_result_message)

    return messages

def output_claude_converse(messages, model='haiku', system=None, toolConfig=None, region_name='us-west-2'):
    from botocore.exceptions import ClientError
    try:
        response = claude_converse(messages, model, False, system, toolConfig, region_name)

        output_message = response['output']['message']

        print(f"Role: {output_message['role']}")

        for content in output_message['content']:
            if content.get("text"):
                print(f"Text: {content['text']}")
            elif content.get("toolUse"):
                print(f"toolUseId: {content['toolUse']['toolUseId']}")
                print(f"name: {content['toolUse']['name']}")
                print(f"input: {content['toolUse']['input']}")
        token_usage = response['usage']
        print(f"Input tokens:  {token_usage['inputTokens']}")
        print(f"Output tokens:  {token_usage['outputTokens']}")
        print(f"Total tokens:  {token_usage['totalTokens']}")
        print(f"Stop reason: {response['stopReason']}")  
    
    except ClientError as err:
        message = err.response['Error']['Message']
        print("A client error occurred: %s", message)
    else:
        print(
            f"\nFinished generating text with model {model}.")
    return response


def caculator_output(messages, response):
    output_message = response['output']['message']
    messages.append(output_message)
    if response['stopReason'] == 'tool_use':
        # Tool use requested. Call the tool and send the result to the model.
        tool_requests = response['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                print(f"Requesting tool {tool['name']}. Request: {tool['toolUseId']}")

                if tool['name'] == 'calculator':
                    tool_result = {}
                    try:
                        result = process_tool_call(tool['name'], tool['input'])
                        print(f"result : {result}")

                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"json": {"result": result}}]
                        }
                    except StationNotFoundError as err:
                        print(f"err : {err}")
                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"text":  err.args[0]}],
                            "status": 'error'
                        }

                    tool_result_message = {
                        "role": "user",
                        "content": [
                            {
                                "toolResult": tool_result

                            }
                        ]
                    }
                    messages.append(tool_result_message)
    return messages

We'll define a simple calculator tool that can perform basic arithmetic operations. The tool will take a mathematical expression as input and return the result.
Note that we are calling eval on the outputted expression. This is bad practice and should not be used generally but we are doing it for the purpose of demonstration.
In this example, we define a calculate function that takes a mathematical expression as input, removes any non-digit or non-operator characters using a regular expression, and then evaluates the expression using the built-in eval() function. If the evaluation is successful, the result is returned as a string. If an error occurs during evaluation, an error message is returned.

We then define the calculator tool with an input schema that expects a single expression property of type string.

In [2]:
toolConfig = {
    "tools": [
    {
        "toolSpec": {
            "name": "calculator",
            "description": "기본적인 산술 연산을 수행하는 간단한 계산기",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "평가할 수학 표현식입니다.(예: '2 + 3 * 4')."
                        }
                    },
                    "required": ["expression"]
                }
            }
        }
    }
],
    "toolChoice": {
        "tool" : {
            "name":"calculator"
        }
    }
}
        

In [3]:
%%time
prompt = "1,984,135 * 9,343,116의 결과는?"

message = [{
    "role": "user",
    "content": [{"text": prompt}]
}]
    
# response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)
# message = caculator_output(message, response)
# response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)

response, msg_param = output_claude_converse_stream(message, 'sonnet', toolConfig=toolConfig)
message = caculator_stream_output(message, response, msg_param)
response, msg_param = output_claude_converse_stream(message, 'sonnet', toolConfig=toolConfig)


Role: assistant
{"expression": "1984135 * 9343116"}
Stop reason: tool_use

Token usage
Input tokens: 485
Output tokens: 32
Total tokens: 517
Latency: 1151 milliseconds

Finished generating text with model sonnet.
Requesting tool calculator. Request: tooluse_Ea4D76u4QmyfmRNxFV4KWw
result : 18538003464660

Role: assistant
{"expression": "18538003464660"}
Stop reason: tool_use

Token usage
Input tokens: 566
Output tokens: 32
Total tokens: 598
Latency: 1367 milliseconds

Finished generating text with model sonnet.
CPU times: user 172 ms, sys: 16.3 ms, total: 188 ms
Wall time: 2.72 s


In [4]:
%%time
prompt = "(12851 - 593) * 301 + 76 계산해줘"

message = [{
    "role": "user",
    "content": [{"text": prompt}]
}]

response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)
message = caculator_output(message, response)
response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)

Role: assistant
toolUseId: tooluse_qLlC0QZGQY2cx6W4Fl9OSg
name: calculator
input: {'expression': '(12851 - 593) * 301 + 76'}
Input tokens:  486
Output tokens:  47
Total tokens:  533
Stop reason: tool_use

Finished generating text with model sonnet.
Requesting tool calculator. Request: tooluse_qLlC0QZGQY2cx6W4Fl9OSg
result : 3689734
Role: assistant
toolUseId: tooluse_bRv8fy5dRmWukwRZY07b1w
name: calculator
input: {'expression': '(12851 - 593) * 301 + 76'}
Input tokens:  571
Output tokens:  47
Total tokens:  618
Stop reason: tool_use

Finished generating text with model sonnet.
CPU times: user 33.6 ms, sys: 365 μs, total: 33.9 ms
Wall time: 1.77 s


In [5]:
%%time
prompt = "15910385 을 193053으로 나눈 값은 뭐에요?"

message = [{
    "role": "user",
    "content": [{"text": prompt}]
}]

response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)
message = caculator_output(message, response)
response = output_claude_converse(message, 'sonnet', toolConfig=toolConfig)

Role: assistant
toolUseId: tooluse_cbsKkOoFShW6_803jpkkvQ
name: calculator
input: {'expression': '15910385 / 193053'}
Input tokens:  490
Output tokens:  40
Total tokens:  530
Stop reason: tool_use

Finished generating text with model sonnet.
Requesting tool calculator. Request: tooluse_cbsKkOoFShW6_803jpkkvQ
result : 82.41459599177428
Role: assistant
toolUseId: tooluse_GTIdjttjRmCUL6bQklM1tw
name: calculator
input: {'expression': 'round(15910385 / 193053, 2)'}
Input tokens:  572
Output tokens:  45
Total tokens:  617
Stop reason: tool_use

Finished generating text with model sonnet.
CPU times: user 32.9 ms, sys: 372 μs, total: 33.2 ms
Wall time: 1.81 s
